In [91]:
import pandas as pd
import numpy as np

In [92]:
# get authentication of Tingo Api
import requests

headers = {
        'Content-Type': 'application/json'
        }
requestResponse = requests.get("https://api.tiingo.com/api/test?token=07490fabce04e447391b5861afc0702824eb3008",
                                    headers=headers)
print(requestResponse.json())

{'message': 'You successfully sent a request'}


In [93]:
from datetime import date
from datetime import timedelta 
today = date.today() 
yesterday = today - timedelta(days = 1) 
# return the quarter of a day
def quarter(date):
    if date. month == 3:
        return 1
    if date. month == 6:
        return 2
    if date. month == 9:
        return 3
    if date. month == 12:
        return 4

In [94]:
# current list of S&P 500 from Wikipedia Api
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df = table[0]
sp500_list = pd.concat([df.Symbol,df.loc[:, 'GICS Sector']], axis=1)
pd.set_option('display.max_rows', None)
print(sp500_list);

    Symbol             GICS Sector
0      MMM             Industrials
1      ABT             Health Care
2     ABBV             Health Care
3     ABMD             Health Care
4      ACN  Information Technology
5     ATVI  Communication Services
6     ADBE  Information Technology
7      AMD  Information Technology
8      AAP  Consumer Discretionary
9      AES               Utilities
10     AFL              Financials
11       A             Health Care
12     APD               Materials
13    AKAM  Information Technology
14     ALK             Industrials
15     ALB               Materials
16     ARE             Real Estate
17    ALXN             Health Care
18    ALGN             Health Care
19    ALLE             Industrials
20     LNT               Utilities
21     ALL              Financials
22   GOOGL  Communication Services
23    GOOG  Communication Services
24      MO        Consumer Staples
25    AMZN  Consumer Discretionary
26    AMCR               Materials
27     AEE          

In [140]:
# focus on data for past 3 years
start_date = yesterday - timedelta(days = 1095) 
start = start_date.strftime("%Y-%m-%d")
end = yesterday.strftime("%Y-%m-%d")

In [95]:
# get financial metrics from Tingo Api
def market_cap(ticker, date):
    ticker = ticker.replace(".", "-")
    start_date = date.strftime("%Y-%m-%d")
    headers = {
            'Content-Type': 'application/json'
        }
    # make request
    requestResponse = requests.get("https://api.tiingo.com/tiingo/fundamentals/"
                                       +ticker+"/daily?token=07490fabce04e447391b5861afc0702824eb3008&startDate="
                                       +start_date, headers=headers)
    market_cap_ = requestResponse.json()[0]['marketCap']
    # return market cap of yesterday if today's data is unavailable
    if market_cap_ != None:
        return market_cap_
    else:
        return market_cap_(ticker, date - timedelta(days = 1) )
    
    
def ev(ticker, date):
    ticker = ticker.replace(".", "-")
    start_date = date.strftime("%Y-%m-%d")
    headers = {
            'Content-Type': 'application/json'
        }
    requestResponse = requests.get("https://api.tiingo.com/tiingo/fundamentals/"
                                       +ticker+"/daily?token=07490fabce04e447391b5861afc0702824eb3008&startDate="
                                       +start_date, headers=headers)
    ev_ = requestResponse.json()[0]['enterpriseVal']
    if ev_ != None:
        return ev_
    else:
        return ev(ticker, date - timedelta(days = 1) )

def pe(ticker, date):
    ticker = ticker.replace(".", "-")
    start_date = date.strftime("%Y-%m-%d")
    headers = {
            'Content-Type': 'application/json'
        }
    requestResponse = requests.get("https://api.tiingo.com/tiingo/fundamentals/"
                                       +ticker+"/daily?token=07490fabce04e447391b5861afc0702824eb3008&startDate="
                                       +start_date, headers=headers)
    pe_ = requestResponse.json()[0]['peRatio']
    
    if pe_ != None:
        # make the company 5th if it has negative pe
        if pe_ < 0:
            pe_ = 1000
        return pe_
    else:
        return pe(ticker, date - timedelta(days = 1) )
    
def pb(ticker, date):
    ticker = ticker.replace(".", "-")
    start_date = date.strftime("%Y-%m-%d")
    headers = {
            'Content-Type': 'application/json'
        }
    requestResponse = requests.get("https://api.tiingo.com/tiingo/fundamentals/"
                                       +ticker+"/daily?token=07490fabce04e447391b5861afc0702824eb3008&startDate="
                                       +start_date, headers=headers)
    pb_ = requestResponse.json()[0]['pbRatio']
    if pb_ != None:
        return pb_
    else:
        return pb(ticker, date + timedelta(days = 1) )

def gross_margin(ticker, date):
    ticker = ticker.replace(".", "-")
    headers = {
            'Content-Type': 'application/json'
        }
    requestResponse = requests.get("https://api.tiingo.com/tiingo/fundamentals/"
                                       +ticker+"/statements?token=07490fabce04e447391b5861afc0702824eb3008", headers=headers)
    data = pd.DataFrame(requestResponse.json()) 
    # fix bugs of source data
    if (ticker == 'COST') | (ticker == 'V') | (ticker == 'AAPL'):
        date = date + timedelta(days = 90)
    if (ticker == 'PG') | (ticker == 'MSFT'):
        date = date + timedelta(days = 180)
    if data.year[0] == 2021:
        year = date.year + 1
    else:
        year = date.year
    q = quarter(date)
    target_data = data.loc[(data.year == year) & (data.quarter == q)]
    
    if (ticker == 'JNJ') & (year == 2017):
        if q == 3:
            target_data = data.loc[data.date == '2017-10-01']
        if q == 4:
            target_data = data.loc[data.date == '2017-12-31']
    elif (ticker == 'JNJ') & (year == 2018):
        if q == 3:
            target_data = data.loc[data.date == '2018-9-30']
        if q == 4:
            target_data = data.loc[data.date == '2017-12-30']    
            
    # if the data of current quarter is unavailable, use past data
    while target_data.empty:
        if q == 1:
            q = 4
            year -=1
        else:
            q -= 1
        target_data = data.loc[(data.year == year) & (data.quarter == q)]
            
    statement_data = target_data['statementData'][target_data.index[0]]
    overview = pd.DataFrame(statement_data['overview'])
    target_value = overview.loc[overview.dataCode == 'grossMargin'].value
    gross_margin_ = target_value[target_value.index[0]] 
    
    if gross_margin_ != None:
        return gross_margin_
    else:
        return gross_margin(ticker, date + timedelta(days = 1) )

def roe(ticker, date):
    ticker = ticker.replace(".", "-")
    # quarter data after 2020-03-28 is unavaliable
    headers = {
            'Content-Type': 'application/json'
        }
    requestResponse = requests.get("https://api.tiingo.com/tiingo/fundamentals/"
                                       +ticker+"/statements?token=07490fabce04e447391b5861afc0702824eb3008", headers=headers)
    data = pd.DataFrame(requestResponse.json())
    if data.year[0] == 2021:
        year = date.year + 1
    else:
        year = date.year
    q = quarter(date)
    target_data = data.loc[(data.year == year) & (data.quarter == q)]
    while target_data.empty:
        if q == 1:
            q = 4
            year -=1
        else:
            q -= 1
        target_data = data.loc[(data.year == year) & (data.quarter == q)]
    statement_data = target_data['statementData'][target_data.index[0]]
    overview = pd.DataFrame(statement_data['overview'])
    target_value = overview.loc[overview.dataCode == 'roe'].value
    roe_ = target_value[target_value.index[0]] 
    
    if roe_ != None:
        return roe_
    else:
        return roe(ticker, date + timedelta(days = 1) )

def revenue(ticker, date):
    ticker = ticker.replace(".", "-")
    headers = {
            'Content-Type': 'application/json'
        }
    requestResponse = requests.get("https://api.tiingo.com/tiingo/fundamentals/"
                                       +ticker+"/statements?token=07490fabce04e447391b5861afc0702824eb3008", headers=headers)
    data = pd.DataFrame(requestResponse.json())
    if ticker == 'NKE':
        date = date + timedelta(days = 180) 
    if data.year[0] == 2021:
        year = date.year + 1
    else:
        year = date.year
    q = quarter(date)
    target_data = data.loc[(data.year == year) & (data.quarter == q)]
    while target_data.empty:
        if q == 1:
            q = 4
            year -=1
        else:
            q -= 1
        target_data = data.loc[(data.year == year) & (data.quarter == q)]
    statement_data = target_data['statementData'][target_data.index[0]]
    income_statement = pd.DataFrame(statement_data['incomeStatement'])
    target_value = income_statement.loc[income_statement.dataCode == 'revenue'].value
    revenue_ = target_value[target_value.index[0]] 
   
    if revenue_ != None:
        return revenue_
    else:
        return revenue(ticker, date + timedelta(days = 1) )

def ebitda(ticker, date):
    ticker = ticker.replace(".", "-")
    headers = {
            'Content-Type': 'application/json'
        }
    requestResponse = requests.get("https://api.tiingo.com/tiingo/fundamentals/"
                                       +ticker+"/statements?token=07490fabce04e447391b5861afc0702824eb3008", headers=headers)
    data = pd.DataFrame(requestResponse.json())
    if ticker == 'DIS':
        date = date + timedelta(days = 90) 
    if data.year[0] == 2021:
        year = date.year + 1
    else:
        year = date.year
    q = quarter(date)
    target_data = data.loc[(data.year == year) & (data.quarter == q)]
    while target_data.empty:
        if q == 1:
            q = 4
            year -=1
        else:
            q -= 1
        target_data = data.loc[(data.year == year) & (data.quarter == q)]
    statement_data = target_data['statementData'][target_data.index[0]]
    income_statement = pd.DataFrame(statement_data['incomeStatement'])
    target_value = income_statement.loc[income_statement.dataCode == 'ebitda'].value
    ebitda_ = target_value[target_value.index[0]] 
    
    if ebitda_ != None:
        return ebitda_
    else:
        return ebitda(ticker, date + timedelta(days = 1) )

def noi(ticker, date):
    ticker = ticker.replace(".", "-")
    headers = {
            'Content-Type': 'application/json'
        }
    requestResponse = requests.get("https://api.tiingo.com/tiingo/fundamentals/"
                                       +ticker+"/statements?token=07490fabce04e447391b5861afc0702824eb3008", headers=headers)
    data = pd.DataFrame(requestResponse.json())
    if data.year[0] == 2021:
        year = date.year + 1
    else:
        year = date.year
    q = quarter(date)
    target_data = data.loc[(data.year == year) & (data.quarter == q)]
    while target_data.empty:
        if q == 1:
            q = 4
            year -=1
        else:
            q -= 1
        target_data = data.loc[(data.year == year) & (data.quarter == q)]
    statement_data = target_data['statementData'][target_data.index[0]]
    income_statement = pd.DataFrame(statement_data['incomeStatement'])
    target_income = income_statement.loc[income_statement.dataCode == 'opinc'].value
    target_expense = income_statement.loc[income_statement.dataCode == 'opex'].value
    noi_ = target_income[target_income.index[0]] - target_expense[target_expense.index[0]]
    
    if noi_ != None:
        return noi_
    else:
        return noi(ticker, date + timedelta(days = 1) )


In [139]:
# print top company list ranked by the ratio for the sector 
def daily_top_company(sector, ratio):
    calendar_dates = pd.date_range(start=start, end=end, freq='Q')
        
    daily_top_company = pd.DataFrame(index = calendar_dates, columns = pd.Series(['1st', '2nd', '3rd', '4th', '5th']))
    MarketCap = pd.Series(np.random.normal(0, 5, len(sp500_list)), index = sp500_list.index , name='MarketCap')
    i = 0
    # loop for every company
    while i < 505: 
        # some tickers are different form the ones in the api but they do not effect final result
        if (i == 23) or (i == 193) or (i == 204) or (i == 338 ) or (i == 457):
            MarketCap[i] = 0
            i += 1
        # only care about companies in specific sector
        elif sp500_list.loc[:, 'GICS Sector'][i] == sector:
            ticker = sp500_list.Symbol[i]
            ticker = ticker.replace("DISCK", "DISCA")
            MarketCap[i] = market_cap(ticker, yesterday)
            i += 1
        else:
            MarketCap[i] = 0
            i += 1
    # sort the list by market cap
    total_list = pd.concat([sp500_list.Symbol, sp500_list.loc[:, 'GICS Sector'], MarketCap], axis = 1)
    sorted_list = total_list.sort_values(by = ['MarketCap'])
    sorted_list.index = total_list.index
    # extract the top 5 and reindex
    top_company =  sorted_list.Symbol.tail(5)
    standard_index = pd.Series([0, 1, 2, 3, 4])
    top_company.index = standard_index
    top_company.name = 'top_company'
        
                              
     # loop for every day in past 3 years                        
    it = 0
    while it < len(calendar_dates):   
        date = calendar_dates[it]
        # get the financial metrics for top 5 
        ratios = pd.Series(np.random.normal(0, 5, 5), index = standard_index , name='ratios')
        j = 0 
        while j < 5:
        
            if ratio == 'pb':
                ratios[j] = pb(top_company[j], date)
            if ratio == 'pe':
                ratios[j] = pe(top_company[j], date)
            if  ratio == 'roe':
                ratios[j] = roe(top_company[j], date)
            if  ratio == 'noi':
                ratios[j] = noi(top_company[j], date)
            if  ratio == 'revenue':
                ratios[j] = revenue(top_company[j], date)
            if  ratio == 'gross_margin':
                ratios[j] = gross_margin(top_company[j], date)
            if  ratio == 'ev/ebitda':
                if ebitda(top_company[j], date) != 0:
                    ratios[j] = ev(top_company[j], date) / ebitda(top_company[j], date)
                else:
                    ratios[j] = 0
            
            j += 1
            
        # rank top 5 by the metric
        ratio_list = pd.concat([top_company, ratios], axis = 1)
        sorted_ratio_list = ratio_list.sort_values(by = ['ratios'])
        # higher metric is unattractive
        if (ratio == 'pb') or (ratio == 'pe') or (ratio == 'ev/ebitda'):
            sorted_ratio_list.index = ratio_list.index
        else:
            # lower metric is unattractive
            new_index = pd.Series([4, 3, 2, 1, 0])
            sorted_ratio_list.index = new_index
            sorted_ratio_list = sorted_ratio_list.sort_index()
        sorted_top_company = sorted_ratio_list.top_company
        
        # fill the daily top 5 company list
        daily_top_company['1st'][calendar_dates[it]] = sorted_top_company[0]
        daily_top_company['2nd'][calendar_dates[it]] = sorted_top_company[1]
        daily_top_company['3rd'][calendar_dates[it]] = sorted_top_company[2]
        daily_top_company['4th'][calendar_dates[it]] = sorted_top_company[3]
        daily_top_company['5th'][calendar_dates[it]] = sorted_top_company[4]
        it += 1
    
    print(daily_top_company)

In [141]:
sector = 'Communication Services'
ratio = 'ev/ebitda'
daily_top_company(sector, ratio)

           1st   2nd    3rd    4th    5th
2017-09-30   T    VZ   NFLX  GOOGL     FB
2017-12-31  VZ  NFLX      T     FB  GOOGL
2018-03-31   T    VZ  GOOGL   NFLX     FB
2018-06-30   T    VZ   NFLX     FB  GOOGL
2018-09-30   T    VZ     FB  GOOGL   NFLX
2018-12-31   T    FB     VZ   NFLX  GOOGL
2019-03-31   T    VZ   NFLX  GOOGL     FB
2019-06-30   T    VZ  GOOGL   NFLX     FB
2019-09-30  VZ     T   NFLX     FB  GOOGL
2019-12-31   T    VZ   NFLX     FB  GOOGL
2020-03-31   T    VZ   NFLX     FB  GOOGL
2020-06-30   T    VZ   NFLX     FB  GOOGL


In [142]:
sector = 'Consumer Discretionary'
ratio = 'revenue'
daily_top_company(sector, ratio)

             1st 2nd  3rd  4th  5th
2017-09-30  AMZN  HD  LOW  NKE  MCD
2017-12-31  AMZN  HD  LOW  NKE  MCD
2018-03-31  AMZN  HD  LOW  NKE  MCD
2018-06-30  AMZN  HD  LOW  NKE  MCD
2018-09-30  AMZN  HD  LOW  NKE  MCD
2018-12-31  AMZN  HD  LOW  NKE  MCD
2019-03-31  AMZN  HD  LOW  NKE  MCD
2019-06-30  AMZN  HD  LOW  NKE  MCD
2019-09-30  AMZN  HD  LOW  NKE  MCD
2019-12-31  AMZN  HD  LOW  NKE  MCD
2020-03-31  AMZN  HD  LOW  NKE  MCD
2020-06-30  AMZN  HD  LOW  NKE  MCD


In [143]:
sector = 'Consumer Staples'
ratio = 'gross_margin'
daily_top_company(sector, ratio)

           1st  2nd 3rd  4th   5th
2017-09-30  KO  PEP  PG  WMT  COST
2017-12-31  KO  PEP  PG  WMT  COST
2018-03-31  KO  PEP  PG  WMT  COST
2018-06-30  KO  PEP  PG  WMT  COST
2018-09-30  KO  PEP  PG  WMT  COST
2018-12-31  KO  PEP  PG  WMT  COST
2019-03-31  KO  PEP  PG  WMT  COST
2019-06-30  KO  PEP  PG  WMT  COST
2019-09-30  KO  PEP  PG  WMT  COST
2019-12-31  KO  PEP  PG  WMT  COST
2020-03-31  KO  PEP  PG  WMT  COST
2020-06-30  KO  PEP  PG  WMT  COST


In [127]:
sector = 'Energy'
ratio = 'ev/ebitda'
daily_top_company(sector, ratio)

            1st  2nd  3rd  4th  5th
2017-09-30  COP  PSX  CVX  XOM  KMI
2017-12-31  COP  XOM  CVX  PSX  KMI
2018-03-31  COP  CVX  XOM  KMI  PSX
2018-06-30  COP  PSX  CVX  XOM  KMI
2018-09-30  COP  CVX  PSX  XOM  KMI
2018-12-31  PSX  COP  CVX  XOM  KMI
2019-03-31  COP  CVX  XOM  KMI  PSX
2019-06-30  COP  PSX  CVX  XOM  KMI
2019-09-30  COP  CVX  XOM  PSX  KMI
2019-12-31  COP  XOM  CVX  KMI  PSX
2020-03-31  PSX  CVX  XOM  KMI  COP
2020-06-30  PSX  CVX  XOM  KMI  COP


In [128]:
sector = 'Financials'
ratio = 'pb'
daily_top_company(sector, ratio)  

           1st  2nd    3rd    4th    5th
2017-09-30   C  BAC    JPM    WFC  BRK.B
2017-12-31   C  BAC  BRK.B    WFC    JPM
2018-03-31   C  BAC    WFC  BRK.B    JPM
2018-06-30   C  BAC  BRK.B    WFC    JPM
2018-09-30   C  BAC    WFC  BRK.B    JPM
2018-12-31   C  BAC    WFC    JPM  BRK.B
2019-03-31   C  BAC    WFC    JPM  BRK.B
2019-06-30   C  BAC    WFC  BRK.B    JPM
2019-09-30   C  BAC    WFC  BRK.B    JPM
2019-12-31   C  WFC    BAC  BRK.B    JPM
2020-03-31   C  WFC    BAC  BRK.B    JPM
2020-06-30   C  WFC    BAC    JPM  BRK.B


In [129]:
sector = 'Health Care'
ratio = 'gross_margin'
daily_top_company(sector, ratio)

            1st  2nd  3rd  4th  5th
2017-09-30  PFE  MRK  JNJ  ABT  UNH
2017-12-31  PFE  MRK  JNJ  ABT  UNH
2018-03-31  PFE  MRK  JNJ  ABT  UNH
2018-06-30  PFE  MRK  JNJ  ABT  UNH
2018-09-30  PFE  JNJ  MRK  ABT  UNH
2018-12-31  PFE  MRK  JNJ  ABT  UNH
2019-03-31  PFE  MRK  JNJ  ABT  UNH
2019-06-30  PFE  MRK  JNJ  ABT  UNH
2019-09-30  PFE  MRK  JNJ  ABT  UNH
2019-12-31  PFE  MRK  JNJ  ABT  UNH
2020-03-31  PFE  MRK  JNJ  ABT  UNH
2020-06-30  PFE  MRK  JNJ  ABT  UNH


In [130]:
sector = 'Industrials'
ratio = 'roe'
daily_top_company(sector, ratio)

            1st  2nd  3rd  4th  5th
2017-09-30  UPS  LMT  HON  UNP   BA
2017-12-31   BA  UPS  LMT  UNP  HON
2018-03-31   BA  LMT  UPS  UNP  HON
2018-06-30   BA  LMT  UPS  UNP  HON
2018-09-30   BA  LMT  UPS  UNP  HON
2018-12-31  LMT  UPS  HON  UNP   BA
2019-03-31  LMT  UPS  HON  UNP   BA
2019-06-30  LMT  UPS  HON  UNP   BA
2019-09-30  LMT  UPS  HON  UNP   BA
2019-12-31  LMT  UPS  HON  UNP   BA
2020-03-31  LMT  UPS   BA  HON  UNP
2020-06-30  LMT  UPS   BA  UNP  HON


In [116]:
sector = 'Information Technology'
ratio = 'gross_margin'
daily_top_company(sector, ratio)

           1st 2nd   3rd   4th   5th
2017-09-30  MA   V  MSFT  NVDA  AAPL
2017-12-31  MA   V  NVDA  MSFT  AAPL
2018-03-31  MA   V  MSFT  NVDA  AAPL
2018-06-30  MA   V  MSFT  NVDA  AAPL
2018-09-30  MA   V  MSFT  NVDA  AAPL
2018-12-31  MA   V  MSFT  NVDA  AAPL
2019-03-31  MA   V  MSFT  NVDA  AAPL
2019-06-30  MA   V  MSFT  NVDA  AAPL
2019-09-30  MA   V  MSFT  NVDA  AAPL
2019-12-31  MA   V  MSFT  NVDA  AAPL
2020-03-31  MA   V  MSFT  NVDA  AAPL
2020-06-30  MA   V  MSFT  NVDA  AAPL


In [131]:
sector = 'Materials'
ratio = 'pe'
daily_top_company(sector, ratio)

            1st  2nd  3rd  4th  5th
2017-09-30  APD   DD  SHW  ECL  NEM
2017-12-31  APD   DD  ECL  SHW  NEM
2018-03-31  APD   DD  SHW  ECL  NEM
2018-06-30   DD  SHW  ECL  APD  NEM
2018-09-30   DD  SHW  APD  ECL  NEM
2018-12-31   DD  APD  ECL  SHW  NEM
2019-03-31   DD  APD  ECL  SHW  NEM
2019-06-30   DD  APD  SHW  ECL  NEM
2019-09-30  NEM  APD  SHW  ECL   DD
2019-12-31  NEM  APD  ECL  SHW   DD
2020-03-31  NEM  APD  SHW  ECL   DD
2020-06-30  NEM  APD  SHW  ECL   DD


In [132]:
sector = 'Real Estate'
ratio = 'noi'
daily_top_company(sector, ratio)

             1st   2nd   3rd   4th  5th
2017-09-30   AMT   PLD  EQIX   CCI  DLR
2017-12-31   PLD  EQIX   CCI   DLR  AMT
2018-03-31   PLD  EQIX   CCI   DLR  AMT
2018-06-30   PLD   AMT  EQIX   CCI  DLR
2018-09-30  EQIX   AMT   PLD   CCI  DLR
2018-12-31   PLD  EQIX   CCI   DLR  AMT
2019-03-31   PLD   AMT  EQIX   CCI  DLR
2019-06-30   AMT  EQIX   PLD   CCI  DLR
2019-09-30   AMT   PLD   CCI  EQIX  DLR
2019-12-31   PLD   AMT  EQIX   DLR  CCI
2020-03-31   PLD   AMT  EQIX   CCI  DLR
2020-06-30   PLD   AMT  EQIX   CCI  DLR


In [133]:
sector = 'Utilities'
ratio = 'pe'
daily_top_company(sector, ratio)

            1st  2nd  3rd  4th  5th
2017-09-30  NEE    D  DUK  AEP   SO
2017-12-31  NEE  AEP    D  DUK   SO
2018-03-31  NEE    D  AEP  DUK   SO
2018-06-30  NEE    D  DUK  AEP   SO
2018-09-30  NEE    D  AEP   SO  DUK
2018-12-31  NEE    D  AEP   SO  DUK
2019-03-31  NEE    D  AEP   SO  DUK
2019-06-30   SO  AEP  DUK  NEE    D
2019-09-30   SO  DUK  AEP  NEE    D
2019-12-31   SO  DUK  AEP  NEE    D
2020-03-31  DUK   SO  AEP  NEE    D
2020-06-30   SO  DUK  AEP  NEE    D
